In [7]:
import delta
from pathling import PathlingContext
from spark_bi.spark import FutPathlingContext

from pathlib import Path


pc = PathlingContext.create()
delta_lake = pc.read.delta(str(Path.home() / "data" / "fut-bi-delta-lake"))

25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function display replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function member_of replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function subsumes replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function designation replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function property_string replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function property_code replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function property_integer replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function property_boolean replaced a previously registered function.
25/12/11 14:05:20 WARN SimpleFunctionRegistry: The function property_deci

In [8]:
from pyspark.sql.functions import when, col

organizations = delta_lake.view(
    resource="Organization",
    select=[
        {"column": [{"name": "organization_id", "path": "getResourceKey()"}]},
        {
            "column": [
                {
                    "name": "municipality",
                    "path": "Organization.extension('http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-organization-municipalityCode').valueString",
                }
            ]
        },
    ],
).withColumn(
    "organization_id",
    when(col("organization_id") == "Organization/-1294776313", "Organization/38806").otherwise(
        col("organization_id")
    ),
)


25/12/11 14:05:21 WARN Collection: Traversing a choice element `valueString` without using ofType() is not portable and may not work in some FHIRPath implementations. Consider using ofType() to specify the type of element you want to traverse.


In [9]:
organizations.toPandas()

25/12/11 14:05:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,organization_id,municipality
0,Organization/-1294776311,0410
1,Organization/38806,0159
2,Organization/-1294776323,0173
3,Organization/-1294776324,0217
4,Organization/-1294777375,0561
...,...,...
155758,Organization/-1294967176,0173
155759,Organization/-1294967199,0230
155760,Organization/-1294967220,0851
155761,Organization/-1294967222,0630


In [11]:
careteams = delta_lake.view(
    resource="CareTeam",
    select=[
        {"column": [{"name": "careteam_id", "path": "getResourceKey()"}]},
        {
            "column": [
                {
                    "name": "managingOrganization",
                    "path": "CareTeam.managingOrganization.first().getReferenceKey()",
                }
            ]
        },
    ],
)

In [ ]:
careteams.toPandas()

,careteam_id,managingOrganization
0,CareTeam/-1294777495,None
1,CareTeam/-1294789119,None
2,CareTeam/-1294812443,None
3,CareTeam/-1294813928,None
4,CareTeam/-1294814226,None
5,CareTeam/-1294814229,None
6,CareTeam/-1294814230,None
7,CareTeam/-1294817155,None
8,CareTeam/-1294819235,Organization/29719
9,CareTeam/-1294819236,Organization/38806


In [ ]:
result = careteams.join(
    organizations,
    careteams["managingOrganization"] == organizations["organization_id"],
    how="inner",
)

In [ ]:
materialised = result.toPandas()

In [ ]:
materialised.groupby("municipality").size().reset_index(name="count")

,municipality,count
0,0159,1
